In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table
from baseline_model.optimization_results_processing import *
from data_display.baseline_plots import *
from utility.pyomo_preprocessing import *
from utility.input_data_preprocessing import *
from config import settings
from general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log, duckdb_to_dict

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from data_display.baseline_plots import *
from baseline_model.baseline_input import BaseLineInput
from baseline_model.first_stage.first_stage_pipeline import BaselineFirstStage
from baseline_model.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"
volume_factor = 1e-6

In [2]:
output_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES)) # type: ignore
# smallflex_input_schema: SmallflexInputSchema = SmallflexInputSchema().duckdb_to_schema(file_path=output_file_names["duckdb_input"])

In [3]:
PARALLEL = False
YEARS = [2020, 2021, 2022, 2023]
TURBINE_FACTORS = {0.75, 0.85, 0.95}

SIMULATION_SETTING = [
    {"quantile": 0, "buffer": 0.1, "powered_volume_enabled": True},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "global_price": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False, "global_price": True},
]

YEARS = 2021
TURBINE_FACTORS = 0.85
# hydro_power_mask = c("name").is_in(["Aegina discrete turbine"])
# hydro_power_mask = c("name").is_in(["Aegina discrete turbine", "Aegina pump"])
hydro_power_mask = c("name").is_in(["Aegina discrete turbine", "Aegina continuous turbine", "Aegina pump"])


REAL_TIMESTEP = timedelta(hours=1)
FIRST_STAGE_TIMESTEP = timedelta(days=1)
SECOND_STAGE_TIME_SIM = timedelta(days=3)
ANCILLARY_TIMESTEP = timedelta(hours=4)
TIME_LIMIT = 20 # in seconds
VOLUME_FACTOR = 1e-6

baseline_input: BaseLineInput = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=REAL_TIMESTEP,
    year=YEARS,
    max_alpha_error=2,
    hydro_power_mask =hydro_power_mask,
    volume_factor=VOLUME_FACTOR
)
first_stage: BaselineFirstStage = BaselineFirstStage(
    nb_state= 3,
    input_instance=baseline_input,
    timestep=FIRST_STAGE_TIMESTEP,
    max_turbined_volume_factor=TURBINE_FACTORS
)
first_stage.solve_model()


second_stage: BaselineSecondStage = BaselineSecondStage(
        input_instance=baseline_input, 
        first_stage=first_stage, 
        sim_timestep=SECOND_STAGE_TIME_SIM, 
        ancillary_market_timestep=ANCILLARY_TIMESTEP,
        time_limit=TIME_LIMIT,
        model_nb=0,
        nb_state=3,
        is_parallel=PARALLEL,
        **SIMULATION_SETTING[0]
    )
second_stage.solve_model()

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 15/15 [00:01<00:00, 10.49it/s]
Solving second stage optimization model number 0: 100%|█████████████████████████████████████████████████████████████| 122/122 [08:03<00:00,  3.96s/it]


In [ ]:
optimization_summary, combined_results = combine_second_stage_results(
    optimization_results= second_stage.optimization_results,
    powered_volume= second_stage.powered_volume,
    market_price=second_stage.market_price, 
    index=second_stage.index, 
    flow_to_vol_factor= second_stage.real_timestep.total_seconds() * second_stage.volume_factor)


{'start_basin_volume': shape: (246, 3)
 ┌─────┬────────┬────────────────────┐
 │ B   ┆ sim_nb ┆ start_basin_volume │
 │ --- ┆ ---    ┆ ---                │
 │ u32 ┆ i32    ┆ f64                │
 ╞═════╪════════╪════════════════════╡
 │ 0   ┆ 0      ┆ 14.8               │
 │ 1   ┆ 0      ┆ 0.0                │
 │ 0   ┆ 1      ┆ 14.729618          │
 │ 1   ┆ 1      ┆ 0.089509           │
 │ 0   ┆ 2      ┆ 14.668507          │
 │ …   ┆ …      ┆ …                  │
 │ 1   ┆ 120    ┆ 1.0                │
 │ 0   ┆ 121    ┆ 13.823168          │
 │ 1   ┆ 121    ┆ 0.518932           │
 │ 0   ┆ 122    ┆ 13.647564          │
 │ 1   ┆ 122    ┆ 0.002938           │
 └─────┴────────┴────────────────────┘,
 'remaining_volume': shape: (369, 3)
 ┌─────┬────────┬──────────────────┐
 │ H   ┆ sim_nb ┆ remaining_volume │
 │ --- ┆ ---    ┆ ---              │
 │ u32 ┆ i32    ┆ f64              │
 ╞═════╪════════╪══════════════════╡
 │ 0   ┆ 0      ┆ 0.0              │
 │ 1   ┆ 0      ┆ 0.0              │
 

In [5]:
second_stage.powered_volume.pivot(on="H",index="sim_nb", values="powered_volume")

sim_nb,0,1,2
u32,f64,f64,f64
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.181871,0.177747,0.0
3,0.0,0.088874,0.0
4,0.103016,0.088874,0.0
…,…,…,…
117,0.545613,0.444368,0.0
118,0.545613,0.533242,0.0
119,0.363742,0.355495,0.0


In [6]:
print(extract_optimization_results(first_stage.model_instance, "ancillary_power").to_pandas().to_string())

       T  CH  ancillary_power
0      0   1         0.000000
1      1   1         0.000000
2      2   1         0.000000
3      3   1         0.000000
4      4   1         0.000000
5      5   1         0.000000
6      6   1         3.500000
7      7   1         3.500000
8      8   1         0.000000
9      9   1         0.000000
10    10   1         3.500000
11    11   1         0.000000
12    12   1         0.000000
13    13   1         3.500000
14    14   1         0.000000
15    15   1         0.000000
16    16   1         0.000000
17    17   1         0.000000
18    18   1         0.000000
19    19   1         0.000000
20    20   1         0.000000
21    21   1         0.000000
22    22   1         0.000000
23    23   1         0.000000
24    24   1         0.000000
25    25   1         0.000000
26    26   1         0.000000
27    27   1         0.000000
28    28   1         0.000000
29    29   1         0.000000
30    30   1         0.000000
31    31   1         0.000000
32    32  

In [7]:
optimization_summary, combined_results = combine_second_stage_results(
    optimization_results= second_stage.optimization_results,
    powered_volume= second_stage.powered_volume,
    market_price=second_stage.market_price, 
    index=second_stage.index, 
    flow_to_vol_factor= second_stage.real_timestep.total_seconds() * second_stage.volume_factor)

In [8]:
plot_first_stage_result(
    simulation_results=first_stage.optimization_results, 
    time_divider=7
).show()

In [9]:
first_stage.water_flow_factor

B,H,BH,water_factor
i64,u32,list[i64],i64
0,0,"[0, 0]",-1
0,1,"[0, 1]",-1
0,2,"[0, 2]",1
1,0,"[1, 0]",1
1,1,"[1, 1]",1
1,2,"[1, 2]",-1


In [10]:
first_stage.optimization_results

T,timestamp,market_price,max_market_price,min_market_price,basin_volume_0,basin_volume_1,powered_volume_0,powered_volume_1,powered_volume_2,hydro_power_0,hydro_power_1,hydro_power_2,income
u32,"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2021-01-01 00:00:00 UTC,48.060417,58.3,38.43,0.811261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-02 00:00:00 UTC,55.350833,64.79,42.04,0.811787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-03 00:00:00 UTC,48.8825,60.93,34.87,0.81231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-04 00:00:00 UTC,60.326667,71.94,40.86,0.812833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-05 00:00:00 UTC,58.7175,67.3,44.81,0.81335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
360,2021-12-27 00:00:00 UTC,199.95125,248.0,101.43,0.761055,0.826624,0.0,0.0,0.133122,0.0,0.0,-7.390717,638.827851
361,2021-12-28 00:00:00 UTC,186.126667,224.55,129.61,0.769069,0.693503,0.0,0.0,0.173376,0.0,0.0,-9.625566,774.476112
362,2021-12-29 00:00:00 UTC,165.653333,203.22,90.47,0.779277,0.520127,0.0,0.0,0.173376,0.0,0.0,-9.625566,689.286236


In [11]:
extract_optimization_results(
            model_instance=first_stage.model_instance, var_name="spilled_volume"
        )


T,B,spilled_volume
u32,u32,f64
0,0,0.0
0,1,0.0
1,0,0.0
1,1,0.0
2,0,0.0
…,…,…
362,1,0.0
363,0,0.0
363,1,0.0


In [12]:

turbined_power = extract_optimization_results(
        model_instance=model_instance, var_name="turbined_power"
    )

turbined_power = pivot_result_table(
    df = turbined_power, on="H", index=["T"],
    values="turbined_power")

simulation_results: pl.DataFrame = market_price\
    .join(basin_volume, on = "T", how="inner")\
    .join(turbined_volume, on = "T", how="inner")\
    .join(pumped_volume, on = "T", how="inner")\
    .join(pumped_power, on = "T", how="inner")\
    .join(turbined_power, on = "T", how="inner")\
    .with_columns(
        ((
            pl.sum_horizontal(cs.starts_with("turbined_power")) -
            pl.sum_horizontal(cs.starts_with("pumped_power"))
        ) * c("T").replace_strict(nb_hours_mapping, default=None) * c("market_price")).alias("income")
    )
    
hydro_name = list(map(str, list(model_instance.H))) # type: ignore

simulation_results = simulation_results.with_columns(
    pl.struct(cs.ends_with(hydro) & ~cs.starts_with("basin_volume"))
    .pipe(remove_suffix).alias("hydro_" + hydro) 
    for hydro in hydro_name
).select(    
    ~(cs.starts_with("turbined") | cs.starts_with("pumped")) # type: ignore
)


NameError: name 'model_instance' is not defined